In [1]:
import sys
sys.path.append('../src')
from gow.utilities import load_ucr_dataset, run_knn
from gow import gow_sinkhorn_autoscale
import numpy as np
from ot import dist
import os

ImportError: cannot import name 'UltraTWD_GD' from 'op_tree' (c:\Users\Admin\Downloads\GOW-main\GOW-main\examples\../src\op_tree\__init__.py)

In [2]:
# Get UCR train and test sets
X_train, y_train, X_test, y_test = load_ucr_dataset("../data/UCR", "Chinatown")

Successfully loaded dataset: Chinatown
Size of train data: 20
Size of test data: 343


In [41]:
def add_positional_encoding(X):
    """Add positional encoding to the time series data."""
    n_samples = X.shape[0]
    series_length = X.shape[1]
    new_X = []
    for i in range(n_samples):
        median = np.median(X[i, :, 0])
        for t in range(series_length):
            new_X.append([X[i, t, 0], median* t / series_length])
    return np.array(new_X)
X = add_positional_encoding(np.concatenate((X_train, X_test), axis=0))

In [42]:
import torch
from op_tree.class_ultratwd_gd import UltraTWD_GD  

device = "cuda" if torch.cuda.is_available() else "cpu"
folder = f"UltraTWD_gradient_descent_results"
os.makedirs(folder, exist_ok=True)


In [1]:
model_path = f"{folder}/ultratwd_gradient_iter"
print(f"  Initializing the UltraTWD-GD on {device}...")
model = UltraTWD_GD(X, model_path, device=device, lambda_cost=1.0)  # lambda_cost là tham số cho compute_new_cost
opt = torch.optim.Adam([model.param], lr=1)
log = model.optimize_ultrametric_by_gradient_descent(
    opt, max_iter=800
)

NameError: name 'folder' is not defined

In [29]:
N = X_train.shape[0]        # n_sequences
L = X_train.shape[1]        # series_length (giả sử cố định)
idx1, idx2 = [0], [1]  # các cặp chuỗi muốn so sánh

twd, tsec = UltraTWD_GD.compute_twd_distance_for_sequences(
    subtree=model.subtree,
    param=model.param,
    parents=model.parents,
    n_sequences=N,
    series_length=L,
    idx1=idx1,
    idx2=idx2
)
print(twd)
from dtw import dtw_mean_per_step
d = dtw_mean_per_step(X_train[0].T, X_train[1].T)
print(d)

[157.47354908]
(69.29629629629629, 27)
